# TODO: Add the other images to the appendix.

# A Brief Peek into the Chess Scene in America using Data Science!

by Michael Omori

Here I present my work and how I went about it. I also have slides I presented for anybody who wants to cut to the chase. Or you can read my conclusion. My repo is a jungle, so I just copy snapshots here, also so I don't have to rerun code. If you want to contact me, you can email me at omorim@hawaii.edu.

## Table of Contents

Chess in the US <br>
My own journey in chess and data science <br>
Questions <br>
Architecture Flowchart <br>
Scraping <br>
Database <br>
Cleaning <br>
Analysis and Visualizations <br>
Modeling <br>
Conclusions <br>
Future Work <br>
Appendix

## Chess in the US

A long history.

## My own journey in chess and data science

Where to start. I'll post more on michaeljomori.com if you're interested.

## Architecture Flowchart

<img src="images/architecture.png" width="600">

## Scraping

I get user data from uschess.org Below is an example.

<img src="images/user_site.png" width="600">

<img src="images/user_site2.png" width="400">

My UID is 12852397 with a link of http://www.uschess.org/msa/MbrDtlMain.php?12852397. We can get another user's information by replacing my UID with their UID like http://www.uschess.org/msa/MbrDtlMain.php?11111111. This is a non-existent user. I don't have a list of UIDs so I do guess and check. I try binary search on the UIDs assuming there is some range of values and continuity to get a lower and upper bound

The lowest id: 12435021 <br>
The highest id: 12939951 <br>
Then I do random sampling without replacement when fetching the URLs. If you are attentive, you may notice my overall ranking is out of 67901, which means there are 67901 active chess players. The difference in highest id and lowest id on the otherhand is 504,930. While when I scraped my data, I only got about 300,000 users' data. This is partly because the UIDs are not strictly continuous.

Scraping events was also uncertain in that I don't know how many tournaments a user has played in. <br>
Tournaments can span multiple pages like such: <br>
http://www.uschess.org/msa/MbrDtlTnmtHst.php?12852397.2 <br>
http://www.uschess.org/msa/MbrDtlTnmtHst.php?12852397.3 <br>
So I just keep scraping until I hit some empty event using a while loop and have it return a non-zero value in certain cases. I also wrote my own get_event_ratings function and you can look at my github code if you're interested.

<img src="images/event_loop.png" width="600">

I do some manual parsing, but also do some regex like such:

<img src="images/regex.png" width="600">

Here is an example of what the events looks like before I transform it.

<img src="images/events_df.png" width="600">

## Database

I use SQLITE for my backend database. SQL Server would be overkill. Below is my basic star schema after cleaning and transformations. No need for snowflake schema. On top of which I create a few views for easier querying

<img src="images/schema.png" width="600">

Users contains: UID, State, Gender <br>
Fact table contains: UID, event_id, rating_before, rating_after <br>
Events_dim_cleaned contains: event_id, event_name, date, state <br>
I use text for many of these attributes because sometimes the values will be "NA" or have a string in front of a number like 1000 P(5). So it would be nice to use like an int for example, but this fails a lot. Each table has one primary key and foreign key. And I chose to do inner joins to reduce my data a bit.

Data is 283,785 users. About 30 million events in the fact table. Events are redundant originally when collecting data, so I make a dim table for it called events_dim_cleaned.

## Cleaning

Despite carefully scraping the data, I still get inconsistencies due to the website and my own mistakes. So I have to do some cleaning.

## Analysis and Visualizations

Believe it or not, a few of these visualizations had me scratching my head on how to make: I had to create views and manipulate the data, which was sort of big in the millions. I will mention some things I did, you can look at my code for more deets. I used matplotlib, seaborn, and good o'l Tableau.

<img src="images/number_of_chess_players.jpg" width="800">

Chess has seen an increase in popularity from 1991-2004 at about a rate of 20,000 new people registering each year. But then this sees a sudden plateau afterwards. I'm not sure exactly what the reason for this is, as I would think chess is getting more popular. Maybe the most probable reason is I messed up this graph. So either during the data collection process, I didn't get all the data and get it correctly, or while doing the analysis I messed up. For now, I'll show this, but I will go back and check again. But for now, we'll have to assume the validity of my data and methods. Please bear with me. One more thing is unknown: which is really just the gender is unknown. It could be non-male and non-female, or just the person didn't put in their gender.

<img src="images/max_rating_per_year.jpg" width="800">

The maximum rating for male is the maximum for all, which was acheived by Magnus Carlsen, with a rating of 2882. In this graph, male is under the all. The highest female rating acheived was by Judit Polgar with a rating of 2735.

<img src="images/rating_distributions.jpg" width="800">

Here is the distribution of ratings using histograms and also KDEs. Each of them tends to have a bell-shaped curve; however, there is a peak at 100, possibly because I only include people who play in at least one tournament and there is a lot of people who play in only one tournament and don't play in any more, so there rating stays low.

<img src="images/Dashboard_Players_by_Location.png" width="800">

To determine if chess has seen an increase in popularity, I took a look at the number of chess players in each state. New York hs the highest with 36,401, followed by California, and Texas. Along with Florida and Illinois. Some of the states with not too many people include the mountainous regions such as Idaho and Wyoming. Another interesting one is West Virginia, as pointed out by Professor Mahdi Belcaid. It is surrounded by states that have a fair amount of chess players, but it did not want to follow suit.

<img src="images/Dashboard_Tournaments_by_Location.png" width="800">

Next, I figure having a lot of tournaments is also something a chess player wants. So here's the graph. It looks similar to the one above. With New York, California, and Texas being hotbeds for tournaments. While West Virginia is lackadasical once again. An interesting difference is that there is some more uniformity among the states as there are no states that have very few chess tournaments. Seeing that there is a correlation between the two, I calculated the correlation and got 0.96 linear pearson's correlation.

## Modeling

### Predicting Chess Rating

I used Gender (categorical), State (categorical), and Number of Chess Tournaments (numerical) Played for the features. I used a Catboost model. <br>
Data is 283, 785 users. <br>
RMSE average mean across 5 folds, 80:20 split, 371.16 on training, 371.24 on validation <br>
STDV: 16.890 <br>
RMSE on test is 371.56

### Feature Importance

Features are only so useful as the power of the model. Which in this case, is so so, so the below can be taken with a grain of salt: <br>
1. Number of tournaments played: 53.119 <br>
2. Location: 23.572 <br>
3. Gender: 23.310

## Conclusions

-Chess has seen a rise in popularity especially from 1992 – 2004 <br>
-Chess players haven’t gotten that much stronger in the past 29 years <br>
-The average chess player rating is around 800 <br>
-New York, Texas, and California are great states to live in for chess players <br>
-Chess performance is most impacted by experience, followed by the location of the player, then gender

## Future Work

There are several areas of interest for me. Some include looking at player vs. player statistics. Like how likely is one player to beat another player. Also looking at some of the top players and how fast they improved their rating. Maybe make my Tableau dashboards public so people can interact with them. Overall, I may ask USCF for the complete picture, so all of their data, because I don't think I got quite all of the data.

## Appendix

For those interested in more details, I include some...

### Google Cloud VM

<img src="images/vm_1.png" width="400">

<img src="images/vm_2.png" width="400">

I use Debian 9 for my OS because I like the customizability on it. And a persistent 100GB SSD storage system because it’s faster.
I use a VM because it can stay on forever and I can use it just for this project. I use Google Cloud because it integrates well with Google Colab and Drive. And they provide some free credit.

### Code Structure

My code is here: https://github.com/AstroBoy1/chess-ds

I have a tests folder with simple unit tests. I have a docs folder with a doc to be updated. My main file is scrape_insert.py. Which is a few hundred lines of code. It calls py_sql.py at one point to create a sqlite database and create tables. I have a requirements file for the package versions that need to be installed to prevent any conflicts. I use the standard main import. read_sql.py is used for reading from the db mostly for testing purposes. rating_model.ipynb contains the Catboost code.

### Error Checking

<img src="images/error_checking.png" width="400">

Above is an example of error checking and logging, I do this for my scraping code to ensure it doesn't stop and to make debugging easier.

### Progress Bar

<img src="images/progress_bar.png" width="400">

# Thank you for reading, hope you enjoyed!